# Simulating Argon Gas

This program simulates an Argon Gas.

## Importing packages

Importing the needed packages:

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numba import jit

## Initialization

Initializes the needed variables. These are: <br>
N, the number of particles <br>
T, the temperature. <br>
m, the mass of a Argon particle <br>
rho, the density of the gas <br>
sigma, the interaction radius <br>
epsilon, the interaction strength <br>
h, timestep of the simulation <br>
<br>
The needed variables for each of N particles: position r, momentum p, and force f. These all have 3 components, in the x-, y-, and z-direction.

In [2]:
N = 4
T = 1
m = 1
rho = 1
sigma = 0.1
epsilon = 1
h = 0.1

p = np.zeros((N,3))
r = np.zeros((N,3))
f = np.zeros((N,3))

Now we define a function to set the positions according to a braivais-fcc lattice.

In [3]:
@jit
def initPositions(r,N,rho):
    
    #Compute lenght of sides of cube with volume V=N/rho
    L = (N/rho) ** (1/3)
    
    #Compute laticce constant a that fits all N atoms into V
    K = 1                                                             #Number of unit cell in one direction
    while (4 * (K ** 3)) < N:
        K = K + 1
    a = L/K
    
    #Set position particles in unit cell
    unit_cell = np.array([[0.0, 0.0, 0.0],[0.5*a, 0.5*a, 0.0],[0.5*a, 0.0, 0.5*a],[0.0, 0.5*a, 0.5*a]])
    
    #Place particles on the lattice until N = 4K^3
    n=0                                                               #Number of particles placed in lattice
    for i in range(K):
        for j in range(K):
            for k in range(K):
                for l in range(4):
                    if n<N:
                        r[n,0] = unit_cell[l,0]+i*a
                        r[n,1] = unit_cell[l,1]+j*a
                        r[n,2] = unit_cell[l,2]+k*a
                        n = n + 1
    return r

The momenta according to a Maxwell-Boltzmann distribution, keeping in mind that the particles can have negative velocities along the x-, y-, and z-direction. Subsequently the drift of the center of mass of the particles is shifted to zero and v is scaled to  force the desired temperature upon the gas. We define a function to do so.

In [4]:
@jit
def initMomentum(p,N,T):
    
    #Set the variance of the Gaussian distribution to k*T/m
    k = 1.38e-23
    sigma = np.sqrt(k*T*m)
    mu = 0
    
    #Assign momentum components to all particles
    for n in range(N):
        for j in range(3):
            p[n,j] = np.random.normal(mu,sigma,1)
            
    #Correct for drift of center of mass
    vCM = sum(p)/(N*m)                                         #Velocity of center of mass
    p[:,:] = p[:,:]-vCM*m
    
    return p

## Solve Newton's Equations

We start by defining a function to compute the forces on all particles. To do this we use Lennard-Jones potential for interaction and periodic boundary conditions. The boundary conditions require us to use the minimum image convention.

In [5]:
@jit
def computeForces(r,N,sigma,epsilon):
    L = (N/rho) ** (1/3)
    
    #Compute the distance between all particles
    for i in range(N-1):
        for j in range((i+1),N):
            rij = np.zeros(3)
            for k in range(3):
                rij[k] = r[i,k]-r[j,k]
                #Apply minimum image convention
                if rij[k] > 0.5*L:
                    if rij[k] > 0:
                        rij[k] = rij[k] - L
                    else:
                        rij[k] = rij[k] + L
                        
            #Calculating the force of pair ij
            
            rabs = (rij[0] ** 2 + rij[1] ** 2 + rij[2] ** 2) ** (1/2)           #Absolute distance between pairs
            famp = 24*(epsilon/sigma)*(2*(sigma/rabs)**14 -(sigma/rabs)**8)
            for k in range(3):
                f[i,k] = f[i,k]+rij[k]*famp
                f[j,k] = f[j,k]-rij[k]*famp
                
    return f   

We define a function which executes the Verlet algorithm to find the new position r and momentum p of each particle at a later time t+h.

In [6]:
@jit
def computeTimeStep(r,p,rho,N):
    L = (N/rho) ** (1/3)
    
    computeForces(r,N,sigma,epsilon)
    
    #Now we use the first equation to find the new r
    for i in range(N):
        for k in range(3):
            r[i,k] = r[i,k] + (p[i,k]/m)*h + (1/2)*(f[i,k]/m)*(h**2)
            
            #Place particle back into volume
            if r[i,k] >= L:
                r[i,k] = r[i,k] - L
            elif r[i,k] < 0:
                r[i,k] = r[i,k] + L
            
            #Compute first part of new momentum
            p[i,k] = p[i,k] + (1/2)*f[i,k]*h
    
    #Update the forces to the new positions
    computeForces(r,N,sigma,epsilon)
    
    #Compute the second part of the new momentum
    for i in range(N):
        for k in range(3):
            p[i,k] = p[i,k] + (1/2)*f[i,k]*h
    
    return r,p,f